# Problem


An Excel sheet contains a list of job applications. The second column includes the CVs of applicants. The task is to extract the email IDs from each CV and populate them in the third column of the sheet.

Assumptions: 1.CVs are public as Google Drive links 2.Document format is pdf or docx 3. Supports RFC5322 format mail ids only.


In [1]:
import pandas as pd
import requests
import re
from io import BytesIO
import fitz  # PyMuPDF (for PDFs)
from docx import Document  # For DOCX files

In [2]:
file_path = "D:\Programming\Wootz_Assignment\Applications 2.xlsx"
df = pd.read_excel(file_path)

In [3]:
# Function to convert Google Drive view link to a downloadable link
def convert_to_downloadable_link(view_link):
    try:
        # Check if the view_link is a valid string
        if isinstance(view_link, str) and 'drive.google.com' in view_link:
            file_id = view_link.split('id=')[-1]  # Extract the file ID from the URL
            return f"https://drive.google.com/uc?id={file_id}"
        elif isinstance(view_link, str):
            return view_link  # Return as is if it's already downloadable
        else:
            return "Document not found"  # Handle missing or invalid entries
        
    except Exception as e:
        print(f"Error processing link {view_link}: {e}")
        return "Processing Error"

In [4]:
# Function to check the file type by inspecting its content

def get_file_type(downloadable_link):
    try:
        response = requests.get(downloadable_link, stream=True)
        response.raise_for_status()  # Ensure the request was successful
        
      # Read the first 10 bytes from the file (to check its file type)
        file_header = response.content[:10]
        
        # Check for PDF magic byte(%PDF)
        if file_header[:5] == b'%PDF-':
            return 'PDF'
        
        # Check for DOCX magic bytes (part of the .docx ZIP format signature)
        elif file_header[:2] == b'PK': 
            return 'DOCX'
        
        else:
            print("Unsupported file type (magic bytes don't match).")
            return 'Unsupported'
    
    except Exception as e:
        print(f"Error processing {downloadable_link}: {e}")
        return 'Error'

In [5]:
# Function to extract email from a PDF
def extract_email_from_pdf(pdf_url):
    try:
        # Download the PDF file
        response = requests.get(pdf_url)
        response.raise_for_status()
        
        # Read the PDF content
        pdf_data = BytesIO(response.content)
        document = fitz.open(stream=pdf_data, filetype="pdf")
        text = ""
        
        # Extract text from each page of the PDF
        for page in document:
            text += page.get_text()

        # Find email addresses using regex
        emails = re.findall(r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zAZH]{2,}", text)
        return emails[0] if emails else "N/A"
    except Exception as e:
        print(f"Error processing PDF {pdf_url}: {e}")
        return "Processing Error"

In [6]:
# Function to extract email from a DOCX file
def extract_email_from_docx(docx_url):
    try:
        # Download the DOCX file
        response = requests.get(docx_url)
        response.raise_for_status()
        
        # Read the DOCX content
        docx_data = BytesIO(response.content)
        document = Document(docx_data)
        text = ""
        
        # Extract text from the DOCX
        for paragraph in document.paragraphs:
            text += paragraph.text + "\n"

        # Find email addresses using regex
        emails = re.findall(r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}", text)
        return emails[0] if emails else "N/A"
    except Exception as e:
        print(f"Error processing DOCX {docx_url}: {e}")
        return "Processing Error"


In [7]:
# Function to extract email based on file type (PDF or DOCX)
def extract_email_from_file(url):
    # Convert the URL to a downloadable link
    downloadable_link = convert_to_downloadable_link(url)
    
    # Identify file type by reading first few bytes
    file_type = get_file_type(downloadable_link)
    
    # If it's a PDF, extract the email from the PDF file
    if file_type == 'PDF':
        return extract_email_from_pdf(downloadable_link)
    # If it's a DOCX, extract the email from the DOCX file
    elif file_type == 'DOCX':
        return extract_email_from_docx(downloadable_link)
    # If the file type is unsupported or missing, return an appropriate message
    else:
        return f"Unsupported/Missing file"

In [8]:
# Loop through the rows in column B (second column) to extract email IDs
df['Email ID'] = df['CV'][97:99].apply(extract_email_from_file)

Error processing Document not found: Invalid URL 'Document not found': No scheme supplied. Perhaps you meant https://Document not found?


In [10]:
# Save the updates back to the same file
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df.to_excel(writer, index=False, sheet_name='Sheet1')

    print("Excel file updated with extracted emails.")